In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import os
import sys
import math
import time
from datetime import datetime
import copy
import numpy as np
from scipy.spatial import distance_matrix
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.progress import *

In [2]:
os.getcwd()

'C:\\Users\\Zhifeng Guo\\python_code\\paper3_code\\Benchmark'

In [3]:
def find_most_r_closed_points(dij,k,r):
    N=dij.shape[0]
    min_index=[]
    for i in range(N-k,N):
        row=dij.iloc[i,0:N-k]
        col_1=[col for col in row.nsmallest(n=r, keep='all').index]
        min_index.append(col_1)
    return min_index

def get_y_values(y):
    Yj=[]
    for i in M:
        if y[i].solution_value==1:
            Yj.append(i)
    return Yj

def CPLEX_Model(name,N,M,ML,CL,p,ddfij):
    mdl=Model(name=name,log_output=True)
    #set variables
    x=mdl.binary_var_dict([(i,j) for i in N for j in M],lb=0.0, name="x")
    y=mdl.binary_var_dict([j for j in M],name="y")
    mdl.add_constraints(x[i, j] -y[j]<= 0 for i in N for j in M)
    mdl.add_constraint(mdl.sum(y[j] for j in M) ==p)
    mdl.add_constraints(mdl.sum(x[i, j] for j in M) == 1 for i in N)
    
    mdl.add_constraints(x[row[1], j]+x[row[2], j] -1<= 0  for row in CL.itertuples() for j in M)
    #mdl.add_constraints(x[row[2], j]+x[row[1], j] -1<= 0  for row in CL.itertuples() for j in M)

    mdl.add_constraints(x[row[1], j]-x[row[2], j] == 0  for row in ML.itertuples() for j in M)
    #mdl.add_constraints(x[row[2], j]-x[row[1], j] == 0  for row in ML.itertuples() for j in M)
    
    mdl.minimize(mdl.sum(ddfij[i, j]*x[i,j] for i in N for j in M))
    sol_model = mdl.solve(clean_before_solve=True)
    return x, y, sol_model

In [4]:
file_list=[['bc_datasets/Soybean/soybean.classes.txt',
            'bc_datasets/Soybean/soybean.data.txt',
            ['bc_datasets/Soybean/soybean1.txt','bc_datasets/Soybean/soybean2.txt']],
           
           ['bc_datasets/Protein/protein.classes.txt',
            'bc_datasets/Protein/protein.data.txt',
            ['bc_datasets/Protein/protein1.txt','bc_datasets/Protein/protein2.txt']],
           
           ['bc_datasets/Iris/iris.classes.txt',
            'bc_datasets/Iris/iris.data.txt',
            ['bc_datasets/Iris/iris1.txt','bc_datasets/Iris/iris2.txt']],
           
           ['bc_datasets/Wine/wine.classes.txt',
            'bc_datasets/Wine/wine.data.txt',
            ['bc_datasets/Wine/wine1.txt','bc_datasets/Wine/wine2.txt']],
           
           ['bc_datasets/Ionosphere/ionosphere.classes.txt',
            'bc_datasets/Ionosphere/ionosphere.data.txt',
            ['bc_datasets/Ionosphere/ionosphere1.txt','bc_datasets/Ionosphere/ionosphere2.txt']],
           
           ['bc_datasets/Control/control.classes.txt',
            'bc_datasets/Control/control.data.txt',
            ['bc_datasets/Control/control1.txt','bc_datasets/Control/control2.txt']],
           
           ['bc_datasets/Balance/balance.classes.txt',
            'bc_datasets/Balance/balance.data.txt',
            ['bc_datasets/Balance/balance1.txt','bc_datasets/Balance/balance2.txt']],
           
           ['bc_datasets/Yeast/yeast.classes.txt',
            'bc_datasets/Yeast/yeast.data.txt',
            ['bc_datasets/Yeast/yeast1.txt','bc_datasets/Yeast/yeast2.txt']]
          ]

In [5]:
file_list1=[
           
           ['bc_datasets/Balance/balance.classes.txt',
            'bc_datasets/Balance/balance.data.txt',
            ['bc_datasets/Balance/balance1.txt']]
          ]

In [17]:
results=[]
for files in file_list:
    label_df1=pd.read_csv(files[0],sep=" ",header=None)
    sampled_df1=pd.read_csv(files[1],sep=" ",header=None)
    p=label_df1.value_counts().shape[0]
    sampled_df1.columns = [f'X{i+1}' for i in range(len(sampled_df1.columns))]
    sampled_df1=sampled_df1.dropna(axis=1, how='any')
    df_X=sampled_df1.loc[:, sampled_df1.columns.str.contains('X')]
    n=df_X.shape[0]
    configs=files[2]
    for config in configs:
        print(files[0])
        print(config)
        #print(label_df1.head())
        #print(sampled_df1.head())
        Constraints_data=pd.read_csv(config,sep="\t",header=None)
        SV1=[2*p]
        #SV2=[5*p]
        SV2=[4*p]
        SV3=[6*p]
        SV4=[10*p]
        
        #MV1=[p, p+1, p+2, p+3, p+4]
        MV1=[i for i in range(p,p+2)]
        MV2=[p, p+math.floor(p/2), p+2*math.floor(p/2)]
        MV3=[p, 2*p, 3*p]
        
        #MV1_MV2=MV1+MV2
        
        
        RN=[p]
        #RN=[p,p+1, p+2, p+3, p+4,p+math.floor(p/2), p+2*math.floor(p/2)]
        #RN=[p, p+1, p+2, p+3, p+4]
        
        MV1_with_r_2=[p,p+1,p+2, p+3,p+4]
        #MV1_with_r_2=[60]
        #MV1_with_r_2=[6]
        #MV1_with_r_2=[3, 6]
        #MV1_with_r_2=[i for i in range(p,p+10)]
        #MV1_with_r_2=[p,p+1,p+2,p+3,p+4, 10*p]
        MV2_with_r_2=[p,p+math.floor(p/2), p+2*math.floor(p/2)]
        
        all_columns=[]
        all_columns_1=[]
        all_columns_2=[]
        all_columns_3=[]
        

            
        for k in RN:
            print(k)
            kmeans = KMeans(n_clusters=k, random_state=0).fit(df_X)
            cluster_centers=pd.DataFrame(kmeans.cluster_centers_,columns=df_X.columns, index=["center"+str(i) for i in range(k)])
            frames = [df_X, cluster_centers]
            df_X_1 = pd.concat(frames)
            dfij=pd.DataFrame(distance_matrix(df_X_1.values, df_X_1.values), index=df_X_1.index, columns=df_X_1.index)
            column_3=list(find_most_r_closed_points(dfij,k,r=4))
            #print(column_3)
            #all_columns_3.append(column_3)
            all_columns_3+=column_3
            #print("Here")
            #print(all_columns_3)
            #all_columns=list(set().union(*all_columns_3))
            #print(len(all_columns_3))
            #all_columns_3=all_columns_3[0] 
            #print(all_columns_3)
            #all_columns=list(set().union(*all_columns_3))
            #print(len(all_columns))
            #final_centers=[167, 338, 431]
            #final_centers=[163, 316, 457]
            #final_centers = [44, 98, 250, 454, 647, 877, 1041, 1150, 1175, 1346]
            #final_centers=[77, 98, 250, 351, 516, 557, 651, 776, 877, 1175]
            #final_centers=[55, 65, 85]
            #final_centers=[17, 43, 72]
            #final_centers=[5, 11, 24, 38]
            #final_centers=[85, 107, 205, 243, 307, 367]
            #print(list(set(all_columns).intersection(final_centers)))
        
        
        
        all_columns=all_columns_3
        #all_columns=[[167, 338, 431,163],[  91, 12, 38, 135,65, 316]]
        #all_columns=[[77, 98, 250, 351, 516, 557], [91, 12, 38, 135,65,], [46, 113, 145, 53, 154,85]]
        #print(all_columns)
        #column1=list(set().union(*all_columns[0]))
        column1=list(set().union(*all_columns))
        N=range(sampled_df1.shape[0])
        M=range(len(column1))
        print(M)
        
        Constraints_data.columns = [f'f{i+1}' for i in range(len(Constraints_data.columns))]
        Constraints_data.head()
        ML=Constraints_data[Constraints_data['f3']==1]
        ML = ML.reset_index(drop=True)
        CL=Constraints_data[Constraints_data['f3']==-1]
        
        ddfij=dfij.iloc[0:n,column1]
        print(ddfij.shape)
        ddfij=ddfij.to_numpy()
        objective_value=[]
        print(p)
        start2=datetime.now()
        x, y, sol_model=CPLEX_Model(name="CPLEX_Model",N=N,M=M,ML=ML,CL=CL,p=p,ddfij=ddfij)
        running_time=datetime.now()-start2
        cluster_points=get_y_values(y=y)
        #pd.DataFrame(cluster_points, columns=['cluster_points']).to_csv('results/without_500_20_cluster_points_'+str(k)+'_.csv')
        obj=sol_model.get_objective_value()
        #objective_value.append(obj)
        #times.append(running_time)
        result_each=[obj, running_time]
        results.append(result_each)
        print(running_time)  
        

bc_datasets/Soybean/soybean.classes.txt
bc_datasets/Soybean/soybean1.txt
4
range(0, 16)
(47, 16)
4
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 386 rows and 0 columns.
MIP Presolve modified 192 coefficients.
Aggregator did 32 substitutions.
Reduced MIP has 606 rows, 736 columns, and 2048 nonzeros.
Reduced MIP has 736 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.80 ticks)
Found incumbent of value 215.872742 after 0.00 sec. (6.82 ticks)
Probing time = 0.00 sec. (0.71 ticks)
Tried aggregator 1 time.
Reduced MIP has 606 rows, 736 columns, and 2048 nonzeros.
Reduced MIP has 736 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.43 ticks)
Probing time = 0.00 sec. (0.71 ticks)
Clique table members: 605.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 0.00 sec.

Probing time = 0.00 sec. (1.41 ticks)
Tried aggregator 1 time.
Reduced MIP has 1801 rows, 1740 columns, and 5124 nonzeros.
Reduced MIP has 1740 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (10.12 ticks)
Probing time = 0.00 sec. (1.41 ticks)
Clique table members: 1800.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 0.00 sec. (16.67 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          297.9150        0.0000           100.00%
*     0+    0                          106.2304        0.0000           100.00%
*     0+    0                          105.1216        0.0000           100.00%
*     0     0      integral     0       98.3584       98.3584      383    0.00%
Elapsed time = 0.06 sec. (45.87 ticks, tree = 

   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         1177.1719        0.0000           100.00%
*     0+    0                          836.4412        0.0000           100.00%
*     0     0      integral     0      817.2424      817.2424      733    0.00%
Elapsed time = 0.08 sec. (108.99 ticks, tree = 0.00 MB, solutions = 3)

Root node processing (before b&c):
  Real time             =    0.08 sec. (109.13 ticks)
Parallel b&c, 20 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.08 sec. (109.13 ticks)
0:00:00.167357
bc_datasets/Ionosphere/ionosphere.classes.txt
bc_datasets/Ionosphere/ionosphere2.txt
2
range(0, 8)
(351, 8)
2
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 1061 rows and 0 columns.
MIP Presolve modifi

*     0+    0                         1475.5695     1471.6125             0.27%
      0     0        cutoff           1475.5695     1475.5695       89    0.00%
Elapsed time = 0.51 sec. (852.66 ticks, tree = 0.01 MB, solutions = 4)

Zero-half cuts applied:  168
Lift and project cuts applied:  1
Gomory fractional cuts applied:  3

Root node processing (before b&c):
  Real time             =    0.53 sec. (853.11 ticks)
Parallel b&c, 20 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.53 sec. (853.11 ticks)
0:00:00.776377
bc_datasets/Balance/balance.classes.txt
bc_datasets/Balance/balance2.txt
3
range(0, 15)
(625, 15)
3
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 3574 rows and 0 columns.
MIP Presolve modified 945 coefficients.
Aggregator did 1635 substitutions.
Reduced MIP has

In [18]:
results_df = pd.DataFrame(results, columns = ['opt','time']) 
results_df.to_csv('RN_with_rn_4_without_p_full_revised_beta_2_epsi_002_heuris_results_v_332.csv')